In [1]:
input = readlines("day5_test.txt")

11-element Vector{String}:
 "3-5"
 "10-14"
 "16-20"
 "12-18"
 ""
 "1"
 "5"
 "8"
 "11"
 "17"
 "32"

In [2]:
findfirst(input .== "")

5

In [3]:
range_regex = r"(\d+)-(\d+)"

r"(\d+)-(\d+)"

In [4]:
match(range_regex, input[1]).captures

2-element Vector{Union{Nothing, SubString{String}}}:
 "3"
 "5"

In [5]:
function split_input(input::AbstractVector{<:AbstractString})
    i = findfirst(input .== "")
    range_regex = r"(\d+)-(\d+)"
    ranges = match.(range_regex, input[1:i-1])
    str_2_int = x -> parse(Int, x)
    ranges = [str_2_int(r.captures[1]):str_2_int(r.captures[2]) for r in ranges]
    ids = parse.(Int, input[i+1:end])
    return (ranges, ids)
end

split_input (generic function with 1 method)

In [125]:
ranges, ids = split_input(input)

(UnitRange{Int64}[3:5, 10:14, 16:20, 12:18], [1, 5, 8, 11, 17, 32])

In [7]:
function is_fresh(id::Int, ranges::AbstractVector{<:UnitRange})
    return reduce(+, [id in r for r in ranges]) > 0
end

is_fresh (generic function with 1 method)

In [8]:
[is_fresh(id, ranges) for id in ids] |> sum

3

In [10]:
a = Set()

Set{Any}()

In [11]:
union(a, Set(4,))

Set{Any} with 1 element:
  4

In [12]:
Set(1:4)

Set{Int64} with 4 elements:
  4
  2
  3
  1

In [13]:
function fresh_ids(ranges::AbstractVector{<:UnitRange})
    return reduce((x,y) -> union(Set(x), Set(y)), ranges)
end

fresh_ids (generic function with 1 method)

In [107]:
merge_ranges(2:5, 1:1) .|> length 

2-element Vector{Int64}:
 1
 4

In [121]:
function overlap_ranges(range_a::UnitRange, range_b::UnitRange)
    if range_a[end] in range_b || range_a[1] in range_b
        new_range = min(range_a[1], range_b[1]):max(range_a[end], range_b[end])
        return (new_range, true)
    else
        return (range_a, false)
    end
end

overlap_ranges (generic function with 1 method)

In [127]:
ranges

4-element Vector{UnitRange{Int64}}:
 3:5
 10:14
 16:20
 12:18

In [126]:
accumulate((x,y) -> overlap_ranges(x[1],y), ranges, init=(ranges[1], true))

4-element Vector{Tuple{UnitRange{Int64}, Bool}}:
 (3:5, 1)
 (3:5, 0)
 (3:5, 0)
 (3:5, 0)

In [133]:
map(!, [true, false, false])

3-element Vector{Bool}:
 0
 1
 1

In [207]:
function merge_ranges(ranges::AbstractVector{<:UnitRange}, init_ind::Int=1)
    merges = accumulate((x,y) -> overlap_ranges(x[1],y), ranges, init=(ranges[init_ind], true))
    new_range = merges[end][1]
    eaten = getindex.(merges, 2)
    remaining = ranges[map(!, eaten)]
    return vcat([new_range], remaining)
end

merge_ranges (generic function with 4 methods)

In [208]:
merge_ranges([1:2, 1:3, 1:5])

1-element Vector{UnitRange{Int64}}:
 1:5

In [209]:
merge_ranges([1:2, 1:3, 1:5, 6:10, 1:8])

2-element Vector{UnitRange{Int64}}:
 1:8
 6:10

In [175]:
merge_ranges(merge_ranges([1:2, 1:3, 1:5, 6:10, 1:8]))

1-element Vector{UnitRange{Int64}}:
 1:10

In [210]:
function squeeze_ranges(ranges::AbstractVector{<:UnitRange})
    last_n = length(ranges)
    for _ in 1:length(ranges)
        while true
            last_n = length(ranges)
            ranges = merge_ranges(ranges)
            new_n = length(ranges)
            if (last_n - new_n) == 0 break end
        end
        circshift!(ranges, 1)
    end
    return ranges
end


squeeze_ranges (generic function with 2 methods)

In [211]:
squeeze_ranges(ranges)

2-element Vector{UnitRange{Int64}}:
 3:5
 10:20

In [212]:
function fresh_ids(ranges::AbstractVector{<:UnitRange})
    unique_ids = []
    while length(ranges) > 0
        ranges = squeeze_ranges(ranges)
        push!(unique_ids, ranges[1])
        ranges = ranges[2:end]
    end
    return unique_ids
end

fresh_ids (generic function with 1 method)

In [213]:
fresh_ids(ranges)

2-element Vector{Any}:
 3:5
 10:20

In [214]:
ranges, ids = split_input(readlines("day5_test.txt"));

In [215]:
[is_fresh(id, ranges) for id in ids] |> sum

3

In [216]:
fresh_ids(ranges) .|> length |> sum

14

In [217]:
ranges, ids = split_input(readlines("day5_input.txt"));

In [218]:
[is_fresh(id, ranges) for id in ids] |> sum

868

In [221]:
squeeze_ranges(ranges) .|> length |> sum

354143734113772

In [219]:
fresh_ids(ranges) .|> length |> sum

354143734113772